In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import yfinance as yf
from technical_indicator import RSI_calculator, MACD_calculator, momentum_calculator

# === CONFIG ===
STOCK = "LLY"
INTERVAL = "1d"
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=60)

INITIAL_CASH   = 100000.0
INITIAL_SHARES = 0
BLOCK_SIZE     = 100            # shares per trade
RSI_BUY_LVL    = 40
RSI_SELL_LVL   = 60
STOP_PCT       = 0.2          # 5% stop-loss from lot's entry price
FEE_PER_TRADE  = 0.00
SLIPPAGE_BPS   = 0.0            # e.g., 10 = 0.10%

# === DATA ===
df = yf.download(
    STOCK, start=START_DATE, end=END_DATE,
    interval=INTERVAL, auto_adjust=True, prepost=False, progress=False
)
df.index = pd.to_datetime(df.index)

# === INDICATORS ===
rsi_series = RSI_calculator(df)
macd_line, macd_signal = MACD_calculator(df)   # not used for signals but available
momentum_series = momentum_calculator(df)

ind = pd.concat(
    [df["Open"], df["Close"], rsi_series, macd_line, macd_signal, momentum_series],
    axis=1
).dropna()
ind.columns = ["Open", "Close", "RSI", "MACD", "MACD_Signal", "Momentum"]

# --- Build yesterday's signals; execute at today's OPEN (next-bar execution)
mom_up   = (ind["Momentum"] > 0) | (ind["Momentum"].shift(1) <= 0)
mom_down = (ind["Momentum"] < 0)

buy_signal_prev  = (ind["RSI"] < RSI_BUY_LVL) & mom_up
sell_signal_prev = (ind["RSI"] > RSI_SELL_LVL) | mom_down

buy_exec  = buy_signal_prev.shift(1).fillna(False)
sell_exec = sell_signal_prev.shift(1).fillna(False)

# === BACKTEST with per-lot 5% stop-loss ===
cash   = float(INITIAL_CASH)
shares = int(INITIAL_SHARES)
log = []

# Track lots: list of dicts with {"qty": int, "entry": float}
lots = []
slip = SLIPPAGE_BPS / 10_000.0

opens  = ind["Open"].astype(float)
closes = ind["Close"].astype(float)

for i in range(1, len(ind)):
    ts = ind.index[i]
    px_open = opens.iat[i]

    # 1) STOP-LOSS check first (sell ONE breached lot if any)
    #    If you prefer to sell ALL breached lots, loop until no breaches remain.
    stop_sold = False
    for li, lot in enumerate(lots):
        trigger_price = lot["entry"] * (1.0 - STOP_PCT)
        if px_open <= trigger_price and lot["qty"] > 0:
            qty = min(BLOCK_SIZE, lot["qty"])
            proceeds = qty * px_open * (1 - slip) - FEE_PER_TRADE
            cash  += proceeds
            shares -= qty
            lot["qty"] -= qty
            log.append((ts, "STOP_SELL", qty, px_open, shares, cash, lot["entry"], trigger_price))
            stop_sold = True
            break  # sell only one lot per bar on stop; remove this break to sell all
    if stop_sold:
        continue  # skip discretionary buys/sells this bar after a stop triggers

    # 2) Discretionary BUY
    if buy_exec.iat[i]:
        cost = BLOCK_SIZE * px_open * (1 + slip) + FEE_PER_TRADE
        if cash >= cost:
            cash  -= cost
            shares += BLOCK_SIZE
            lots.append({"qty": BLOCK_SIZE, "entry": px_open})
            log.append((ts, "BUY", BLOCK_SIZE, px_open, shares, cash, px_open, None))
        # else: not enough cash -> skip

    # 3) Discretionary SELL
    elif sell_exec.iat[i] and shares >= BLOCK_SIZE:
        # Sell from oldest lot (FIFO)
        sell_qty = BLOCK_SIZE
        px = px_open * (1 - slip)
        cash += sell_qty * px - FEE_PER_TRADE
        shares -= sell_qty

        # decrement from lots FIFO
        remaining = sell_qty
        for lot in lots:
            if lot["qty"] == 0:
                continue
            used = min(lot["qty"], remaining)
            lot["qty"] -= used
            remaining -= used
            if remaining == 0:
                break

        log.append((ts, "SELL", sell_qty, px_open, shares, cash, None, None))

# Final portfolio stats
last_price = closes.iloc[-1]
final_value = cash + shares * last_price
start_value = INITIAL_CASH + INITIAL_SHARES * closes.iloc[0]
pnl = final_value - start_value
roi = (pnl / start_value) * 100 if start_value else 0.0

# === RESULTS ===
cols = ["Time", "Action", "Qty", "ExecPrice", "Shares_After", "Cash_After", "LotEntry", "StopTrigger"]
trades = pd.DataFrame(log, columns=cols)

print("\n=== SUMMARY ===")
print(f"Start Value:        ${start_value:,.2f}")
print(f"Final Cash:         ${cash:,.2f}")
print(f"Final Shares:       {shares}")
print(f"Last Price:         ${last_price:,.2f}")
print(f"Final Portfolio:    ${final_value:,.2f}")
print(f"P&L:                ${pnl:,.2f}  ({roi:.2f}%)")
print(f"Total Trades:       {len(trades)}")

print("\n=== TRADE LOG (last 10) ===")
print(trades.tail(10).to_string(index=False))


=== SUMMARY ===
Start Value:        $100,000.00
Final Cash:         $31,365.01
Final Shares:       100
Last Price:         $705.17
Final Portfolio:    $101,882.21
P&L:                $1,882.21  (1.88%)
Total Trades:       7

=== TRADE LOG (last 10) ===
      Time Action  Qty  ExecPrice  Shares_After   Cash_After   LotEntry StopTrigger
2025-07-10    BUY  100 789.669983           100 21033.001709 789.669983        None
2025-07-16   SELL  100 773.500000             0 98383.001709        NaN        None
2025-07-17    BUY  100 786.789978           100 19704.003906 786.789978        None
2025-07-30   SELL  100 774.000000             0 97104.003906        NaN        None
2025-07-31    BUY  100 747.299988           100 22374.005127 747.299988        None
2025-08-05   SELL  100 765.570007             0 98931.005859        NaN        None
2025-08-07    BUY  100 675.659973           100 31365.008545 675.659973        None


/var/folders/sr/t9b4x2mn7s35cskh4j9lc0qh0000gn/T/ipykernel_73425/1607527048.py:46: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  buy_exec  = buy_signal_prev.shift(1).fillna(False)
/var/folders/sr/t9b4x2mn7s35cskh4j9lc0qh0000gn/T/ipykernel_73425/1607527048.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sell_exec = sell_signal_prev.shift(1).fillna(False)


In [46]:
ind

,Open,Close,RSI,MACD,MACD_Signal,Momentum
Date,,,,,,
2025-07-08,774.849976,777.659973,18.978033,-7.399020,-6.868701,-29.920044
2025-07-09,783.799988,786.919983,24.000390,-6.420843,-6.779130,-4.320007
2025-07-10,789.669983,790.650024,25.990510,-5.283739,-6.480052,5.619995
2025-07-11,786.349976,793.010010,27.287853,-4.144372,-6.012916,30.280029
2025-07-14,790.000000,799.340027,30.792168,-2.699517,-5.350236,28.700012
2025-07-15,801.669983,771.750000,25.111486,-3.737659,-5.027721,-6.330017
2025-07-16,773.500000,789.799988,33.725686,-3.068538,-4.635884,-2.500000
2025-07-17,786.789978,761.500000,28.240767,-4.766878,-4.662083,-33.619995
2025-07-18,763.859985,771.710022,32.505602,-5.228688,-4.775404,-3.739990


In [51]:
from datetime import datetime, timedelta
import pandas as pd
import yfinance as yf
from technical_indicator import momentum_calculator

# === CONFIG ===
STOCK = "LLY"
INTERVAL = "1m"
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=7)

INITIAL_CASH   = 200_000.0
INITIAL_SHARES = 300
BLOCK_SIZE     = 100            # shares per trade
STOP_PCT       = 0.1          # 5% stop-loss
FEE_PER_TRADE  = 0.00
SLIPPAGE_BPS   = 0.0

# === DATA ===
df = yf.download(
    STOCK, start=START_DATE, end=END_DATE,
    interval=INTERVAL, auto_adjust=True, prepost=False, progress=False
)
df.index = pd.to_datetime(df.index)

# === INDICATOR ===
momentum = momentum_calculator(df)
ind = pd.concat([df["Open"], df["Close"], momentum], axis=1).dropna()
ind.columns = ["Open", "Close", "Momentum"]

# Momentum cross detection
mom_cross_up   = (ind["Momentum"] > 0) & (ind["Momentum"].shift(1) <= 0)
mom_cross_down = (ind["Momentum"] < 0) & (ind["Momentum"].shift(1) >= 0)

# Next-bar execution signals
buy_exec  = mom_cross_up.shift(1).fillna(False)
sell_exec = mom_cross_down.shift(1).fillna(False)

# === BACKTEST ===
cash   = float(INITIAL_CASH)
shares = int(INITIAL_SHARES)
log = []
lots = []
slip = SLIPPAGE_BPS / 10_000.0

opens  = ind["Open"].astype(float)
closes = ind["Close"].astype(float)

for i in range(1, len(ind)):
    ts = ind.index[i]
    px_open = opens.iat[i]

    # 1) STOP-LOSS check first
    stop_sold = False
    for lot in lots:
        trigger_price = lot["entry"] * (1 - STOP_PCT)
        if px_open <= trigger_price and lot["qty"] > 0:
            qty = min(BLOCK_SIZE, lot["qty"])
            proceeds = qty * px_open * (1 - slip) - FEE_PER_TRADE
            cash  += proceeds
            shares -= qty
            lot["qty"] -= qty
            log.append((ts, "STOP_SELL", qty, px_open, shares, cash, lot["entry"], trigger_price))
            stop_sold = True
            break
    if stop_sold:
        continue

    # 2) Momentum BUY
    if buy_exec.iat[i]:
        cost = BLOCK_SIZE * px_open * (1 + slip) + FEE_PER_TRADE
        if cash >= cost:
            cash  -= cost
            shares += BLOCK_SIZE
            lots.append({"qty": BLOCK_SIZE, "entry": px_open})
            log.append((ts, "BUY", BLOCK_SIZE, px_open, shares, cash, px_open, None))

    # 3) Momentum SELL
    elif sell_exec.iat[i] and shares >= BLOCK_SIZE:
        qty = BLOCK_SIZE
        proceeds = qty * px_open * (1 - slip) - FEE_PER_TRADE
        cash += proceeds
        shares -= qty
        remaining = qty
        for lot in lots:
            if lot["qty"] == 0:
                continue
            used = min(lot["qty"], remaining)
            lot["qty"] -= used
            remaining -= used
            if remaining == 0:
                break
        log.append((ts, "SELL", qty, px_open, shares, cash, None, None))

# Final stats
last_price = closes.iloc[-1]
final_value = cash + shares * last_price
start_value = INITIAL_CASH + INITIAL_SHARES * closes.iloc[0]
pnl = final_value - start_value
roi = (pnl / start_value) * 100 if start_value else 0.0

cols = ["Time", "Action", "Qty", "ExecPrice", "Shares_After", "Cash_After", "LotEntry", "StopTrigger"]
trades = pd.DataFrame(log, columns=cols)

print("\n=== SUMMARY ===")
print(f"Start Value:     ${start_value:,.2f}")
print(f"Final Cash:      ${cash:,.2f}")
print(f"Final Shares:    {shares}")
print(f"Last Price:      ${last_price:,.2f}")
print(f"Final Portfolio: ${final_value:,.2f}")
print(f"P&L:             ${pnl:,.2f}  ({roi:.2f}%)")
print(f"Total Trades:    {len(trades)}")

print("\n=== TRADE LOG (last 10) ===")
print(trades.tail(10).to_string(index=False))


=== SUMMARY ===
Start Value:     $389,094.50
Final Cash:      $263,591.13
Final Shares:    200
Last Price:      $705.42
Final Portfolio: $404,675.11
P&L:             $15,580.61  (4.00%)
Total Trades:    211

=== TRADE LOG (last 10) ===
                     Time Action  Qty  ExecPrice  Shares_After    Cash_After   LotEntry StopTrigger
2025-08-15 17:41:00+00:00   SELL  100 703.739990           100 334153.771973        NaN        None
2025-08-15 17:56:00+00:00    BUY  100 703.079895           200 263845.782471 703.079895        None
2025-08-15 18:21:00+00:00   SELL  100 703.626099           100 334208.392334        NaN        None
2025-08-15 18:33:00+00:00    BUY  100 705.479980           200 263660.394287 705.479980        None
2025-08-15 18:47:00+00:00   SELL  100 704.010010           100 334061.395264        NaN        None
2025-08-15 18:51:00+00:00    BUY  100 704.210022           200 263640.393066 704.210022        None
2025-08-15 18:52:00+00:00   SELL  100 704.580017           100 

/var/folders/sr/t9b4x2mn7s35cskh4j9lc0qh0000gn/T/ipykernel_73425/3899822802.py:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  buy_exec  = mom_cross_up.shift(1).fillna(False)
/var/folders/sr/t9b4x2mn7s35cskh4j9lc0qh0000gn/T/ipykernel_73425/3899822802.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sell_exec = mom_cross_down.shift(1).fillna(False)


In [48]:
ind

,Open,Close,Momentum
Datetime,,,
2025-08-08 19:17:00+00:00,159.345001,159.027405,0.807404
2025-08-08 19:18:00+00:00,159.105804,158.830002,0.619995
2025-08-08 19:19:00+00:00,158.875000,158.690002,-0.909698
2025-08-08 19:20:00+00:00,158.872696,158.791702,-1.627304
2025-08-08 19:21:00+00:00,158.820007,158.809998,-1.035004
...,...,...,...
2025-08-15 18:58:00+00:00,149.600006,149.613907,-0.381088
2025-08-15 18:59:00+00:00,149.710007,149.732498,-0.137497
2025-08-15 19:00:00+00:00,149.759995,149.661407,-0.308990


In [49]:
df = df.dropna(subset=["Close","Lower","Upper"])  # after computing bands

cash = INITIAL_CASH
shares = INITIAL_SHARES
log = []

for ts in df.index:
    price = float(df.at[ts, "Close"])
    lower = float(df.at[ts, "Lower"])
    upper = float(df.at[ts, "Upper"])
    slip  = SLIPPAGE_BPS / 10_000

    # Buy if price closes below lower band
    if price < lower:
        cost = BLOCK_SIZE * price * (1 + slip) + FEE_PER_TRADE
        if cash >= cost:
            cash  -= cost
            shares += BLOCK_SIZE
            log.append((ts, "BUY", BLOCK_SIZE, price, shares, cash))

    # Sell if price closes above upper band
    elif price > upper and shares >= BLOCK_SIZE:
        proceeds = BLOCK_SIZE * price * (1 - slip) - FEE_PER_TRADE
        cash  += proceeds
        shares -= BLOCK_SIZE
        log.append((ts, "SELL", BLOCK_SIZE, price, shares, cash))

KeyError: ['Close', 'Lower', 'Upper']